In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [2]:
# Simulamos ser el navegador Chrome:
headers_sesion_chrome={
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "es-ES,es;q=0.9",
"Host": "duckduckgo.com",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
"COOKIE": "ah=es-es; p=-2",
"SEC-FETCH-DEST": "document",
"SEC-FETCH-MODE": "navigate",
"SEC-FETCH-SITE": "none",
"SEC-FETCH-USER": "?1",
"UPGRADE-INSECURE-REQUESTS": "1"
}

# Header googlebot:
headers_googlebot={
"User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",
}

headers=[headers_sesion_chrome, headers_googlebot]

In [3]:
url='https://covid19.isciii.es/'

pag = requests.get(url, headers[0], timeout=10)


In [4]:
soup = BeautifulSoup(pag.content)
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta charset="utf-8" content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="0" http-equiv="Expires"/>
  <title>
   Situación de COVID-19 en España.
  </title>
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   $(document).ready(function() {
$("div .sideNavBox").hide();
$("div #contentBox").css('width', '100%');
    
    $.ajax({
        type: "GET",
        url: "resources/ccaa.csv",
        dataType: "text",
        contentType: "application/json; charset=utf-8",
        success: function(data) {processCCAA(data);
        }
     });
     
     $.ajax({
        type: "GET",
        url: "resources/data.csv",
        dataType: "text",
        contentType: "application/json; charset=utf-8",
 

In [5]:
meses={"enero":1,"febrero":2,"marzo":3,
       "abril":4,"mayo":5,"junio":6,
       "julio":7,"agosto":8,"septiembre":9,
       "octubre":10,"noviembre":11,"diciembre":12}


In [6]:
fecha_act=soup.find("span", id="fecha").string
print(fecha_act)
hora_act=soup.find("span", id="hora").string
print(hora_act)

10 de Marzo de 2020
10:10


In [7]:
# Formatemos la fecha
import re
re_dia=r"[0-9]{2}"
re_mes=r"[a-zA-Z]+"
re_año=r"[0-9]{4}"
fecha_dia=re.findall(re_dia, fecha_act)[0]
fecha_mes=re.findall(re_mes, fecha_act)[1]
fecha_año=re.findall(re_año, fecha_act)[0]
hora=re.findall(re_dia, hora_act)[0]
minuto=re.findall(re_dia, hora_act)[1]

fecha_act=datetime.datetime(int(fecha_año) ,meses[fecha_mes.lower()], int(fecha_dia), int(hora),int(minuto))

In [8]:
tabla_CCAA = soup.find("table")

In [9]:
a_list=list(enumerate(td.string for td in tabla_CCAA.find_all("td")))

In [10]:
dict_ccaa=dict()
for elemento in a_list:
    if elemento[0] // 2 == elemento[0] / 2:
        clave=elemento[1]
    else:
        valor=elemento[1]
        dict_ccaa[clave]=valor
dict_ccaa 

{'Andalucía': '71',
 'Aragón': '38',
 'Principado de Asturias': '22',
 'Islas Baleares': '13',
 'Canarias': '25',
 'Cantabria': '12',
 'Castilla y León': '56',
 'Castilla La Mancha': '39',
 'Cataluña': '124',
 'Galicia': '22',
 'C. Valenciana': '50',
 'Extremadura': '8',
 'Comunidad de Madrid': '782',
 'Región de Murcia': '9',
 'Comunidad Foral de Navarra': '12',
 'País Vasco': '195',
 'La Rioja': '144',
 'Ceuta': '0',
 'Melilla': '0'}

In [11]:
df_act=pd.DataFrame(dict_ccaa, index=[fecha_act])

In [12]:
df_act.to_csv("es_covid_{}.csv".format(datetime.datetime.strftime(fecha_act, format="%Y%m%d")))

NameError: name 'df' is not defined